In [1]:
import torch
import requests
import json
import pandas as pd
from transformers import CamembertTokenizerFast,AutoModel,AutoModelForSequenceClassification
import torch.nn.functional as F
from mlflow.models.signature import infer_signature
from mlflow.transformers import generate_signature_output
import mlflow
from torch import nn
   

c:\Users\sern7\anaconda3\envs\pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
model = torch.load('multilabel.pth',map_location=torch.device('cpu'))


In [4]:
   
BERT_MODEL_NAME = "airesearch/wangchanberta-base-att-spm-uncased"
# model = AutoModel.from_pretrained('multilable_pretrained')
tokenizer = CamembertTokenizerFast.from_pretrained(BERT_MODEL_NAME)

In [5]:
# def predict(text):
#     input = tokenizer(text,padding='max_length', max_length = 256, truncation=True,return_tensors="pt")
#     input = input['input_ids']
#     output = model(input).logits
#     # output = F.sigmoid(output)
    
    
#     return output

In [6]:
text = 'ไม่มีที่วางขยะรอจัดเก็บ วางไว้บนทางเท้า'

In [7]:
# predict(text)

In [8]:
# model.save_pretrained('multilabel_model')

In [9]:
# model = AutoModelForSequenceClassification.from_pretrained('multilabel_model')

In [10]:
model

CamembertForSequenceClassification(
  (roberta): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(25005, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0-11): 12 x CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=Tru

In [11]:
text = 'ไม่มีที่วางขยะรอจัดเก็บ วางไว้บนทางเท้า'

In [12]:
X = tokenizer(text, padding='max_length', max_length=256, truncation=True, return_tensors='pt')['input_ids']

In [13]:
# mlflow.set_tracking_uri('http://127.0.0.1:5000')  # set up connection
# mlflow.set_experiment('multilabel')
# with mlflow.start_run() as run:
#     mlflow.pytorch.save_model(model,'multilabel')

# OverRide output

In [14]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # Define your model architecture here
        self.model = torch.load('multilabel.pth',map_location=torch.device('cpu'))

    def forward(self, x):
        # Define the forward pass of your model
        # This function will be used during training, not for prediction
        # Typically, you would define the layers and operations here
        
        return self.model(x).logits

    def predict(self, x):
        return self.forward(x)

In [15]:
model = MyModel()

In [16]:
model(X)

tensor([[-2.8791,  0.3273, -7.1046, -6.1185, -6.3318,  3.3170, -5.5375, -7.7529,
         -8.0882, -7.0107, -8.0564, -8.9456, -7.5059, -8.0857, -6.1525, -7.6930,
         -7.5997, -8.0280, -8.2484, -8.6904, -7.5230, -7.6227, -8.2611, -9.7334]],
       grad_fn=<AddmmBackward0>)

In [17]:
model.predict(X)

tensor([[-2.8791,  0.3273, -7.1046, -6.1185, -6.3318,  3.3170, -5.5375, -7.7529,
         -8.0882, -7.0107, -8.0564, -8.9456, -7.5059, -8.0857, -6.1525, -7.6930,
         -7.5997, -8.0280, -8.2484, -8.6904, -7.5230, -7.6227, -8.2611, -9.7334]],
       grad_fn=<AddmmBackward0>)

In [18]:
# output =  generate_signature_output(model,X.numpy())
signature = infer_signature(X.numpy(), model.predict(X).detach().numpy())

In [19]:
import mlflow.pytorch
import mlflow

mlflow.set_tracking_uri('http://127.0.0.1:5000')  # set up connection
mlflow.set_experiment('multilabel')
with mlflow.start_run() as run:
    mlflow.pytorch.log_model(model, "model",signature=signature)
model_uri = "runs:/{}/model".format(run.info.run_id)
model = mlflow.pytorch.load_model(model_uri)
model

c:\Users\sern7\anaconda3\envs\pytorch\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


MyModel(
  (model): CamembertForSequenceClassification(
    (roberta): CamembertModel(
      (embeddings): CamembertEmbeddings(
        (word_embeddings): Embedding(25005, 768, padding_idx=1)
        (position_embeddings): Embedding(512, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): CamembertEncoder(
        (layer): ModuleList(
          (0-11): 12 x CamembertLayer(
            (attention): CamembertAttention(
              (self): CamembertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): CamembertSelfOutput(
               

In [22]:
model(X)

tensor([[-2.8791,  0.3273, -7.1046, -6.1185, -6.3318,  3.3170, -5.5375, -7.7529,
         -8.0882, -7.0107, -8.0564, -8.9456, -7.5059, -8.0857, -6.1525, -7.6930,
         -7.5997, -8.0280, -8.2484, -8.6904, -7.5230, -7.6227, -8.2611, -9.7334]],
       grad_fn=<AddmmBackward0>)

In [23]:


def predict_json(server_url, input_json):
    response = requests.post(server_url, json=input_json)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Request failed with status code: %s, response: %s"
                        % (response.status_code, response.text))
    
def predict(server_url, text):
    inputs = tokenizer(text, padding='max_length', max_length=256, truncation=True, return_tensors='pt')['input_ids']
    # print(inputs)
    data = {"inputs":inputs.numpy().tolist()}
    # print(data)
    
    
    
    return predict_json(server_url, data)

text = 'ไม่มีที่วางขยะรอจัดเก็บ วางไว้บนทางเท้า'

predict_result = predict("http://127.0.0.1:1244/invocations", text)

In [24]:
predict_result

{'predictions': [[-2.8791043758392334,
   0.3272898197174072,
   -7.104610919952393,
   -6.118514537811279,
   -6.331795692443848,
   3.317025661468506,
   -5.537487983703613,
   -7.752892971038818,
   -8.088247299194336,
   -7.010658264160156,
   -8.05642318725586,
   -8.945622444152832,
   -7.5058794021606445,
   -8.085744857788086,
   -6.152515411376953,
   -7.692965507507324,
   -7.599690914154053,
   -8.028019905090332,
   -8.24844741821289,
   -8.690380096435547,
   -7.523009777069092,
   -7.622708320617676,
   -8.261131286621094,
   -9.733368873596191]]}

In [25]:
import numpy as np
import torch.nn.functional as F

In [37]:
types = ['ถนน','ทางเท้า','แสงสว่าง','ความปลอดภัย','น้ำท่วม','ความสะอาด','กีดขวาง',
        'ท่อระบายน้ำ','สะพาน','จราจร','สายไฟ','คลอง','เสียงรบกวน','ต้นไม้','ร้องเรียน',
        'ป้าย','สัตว์จรจัด',"PM25",'สอบถาม','เสนอแนะ','คนจรจัด','การเดินทาง','ห้องน้ำ','ป้ายจราจร']

In [44]:
def decode_prediction(predict_result):
    y_pred = []
    res = np.array(predict_result['predictions'])
    res = torch.tensor(res)
    res = F.sigmoid(res)
    for row in res:
        y_pred.append([1 if i>=0.5 else 0 for i in row])

    y_pred_decoded = []
    for i in y_pred:
        tmp = []
        for c in range(len(i)):
            if(i[c]==1):
                tmp.append(types[c])
        y_pred_decoded.append(tmp)

    return y_pred_decoded

In [45]:
decode_prediction(predict_result)

[['ทางเท้า', 'ความสะอาด']]

In [ ]:
# while True:
#     pass

# Textclassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('multilabel_pretrained')

In [ ]:
from transformers import Pipeline


class MyPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        if "maybe_arg" in kwargs:
            preprocess_kwargs["maybe_arg"] = kwargs["maybe_arg"]
        return preprocess_kwargs, {}, {}
    def preprocess(self, inputs):
        return self.tokenizer(inputs, padding='max_length', max_length=256, truncation=True, return_tensors='pt')['input_ids']
        

    def _forward(self, model_inputs):
        return self.model(model_inputs)

    def postprocess(self, model_outputs):
        return model_outputs

In [ ]:
pipeline = MyPipeline(model = model,tokenizer = tokenizer,task = "text-classification")

In [ ]:
from mlflow.models.signature import infer_signature
from mlflow.transformers import generate_signature_output

In [ ]:
text = 'ไม่มีที่วางขยะรอจัดเก็บ วางไว้บนทางเท้า'


In [ ]:
input = tokenizer(text, padding='max_length', max_length=256, truncation=True, return_tensors='pt')['input_ids']

In [ ]:
input.dtype

torch.int64

In [ ]:
input.shape

torch.Size([1, 256])

In [ ]:
res = model(tokenizer(text, padding='max_length', max_length=256, truncation=True, return_tensors='pt')['input_ids']).logits


In [ ]:
res.dtype

torch.float32

In [ ]:
res.shape

torch.Size([1, 24])

In [ ]:
# import numpy as np
# from mlflow.models.signature import ModelSignature
# from mlflow.types.schema import Schema, TensorSpec

# input_schema = Schema(
#     [
#         TensorSpec(np.dtype(np.int64), (1,256)),
#     ]
# )
# output_schema = Schema([TensorSpec(np.dtype(np.float32), (1, 24))])
# signature = ModelSignature(inputs=input_schema, outputs=output_schema)


In [ ]:
pipeline.predict('ไม่มีที่วางขยะรอจัดเก็บ วางไว้บนทางเท้า')

ModelOutput([('logits',
              tensor([[-2.8791,  0.3273, -7.1046, -6.1185, -6.3318,  3.3170, -5.5375, -7.7529,
                       -8.0882, -7.0107, -8.0564, -8.9456, -7.5059, -8.0857, -6.1525, -7.6930,
                       -7.5997, -8.0280, -8.2484, -8.6904, -7.5230, -7.6227, -8.2611, -9.7334]]))])

In [ ]:
# tokenizer = CamembertTokenizerFast.from_pretrained(BERT_MODEL_NAME, padding='max_length', max_length=256, truncation=True, return_tensors='pt')
# tokenizer.padding = 'max_length'  # Pad sequences to the maximum length
# tokenizer.max_length = 256  # Set the maximum sequence length
# tokenizer.truncation = True 
# tokenizer('ไม่มีที่วางขยะรอจัดเก็บ วางไว้บนทางเท้า')

In [ ]:
# tokenizer('ไม่มีที่วางขยะรอจัดเก็บ วางไว้บนทางเท้า', padding='max_length', max_length=256, truncation=True, return_tensors='pt')

In [ ]:
# model(tokenizer('ไม่มีที่วางขยะรอจัดเก็บ วางไว้บนทางเท้า', padding='max_length', max_length=256, truncation=True, return_tensors='pt')['input_ids'])

In [ ]:
text = 'ไม่มีที่วางขยะรอจัดเก็บ วางไว้บนทางเท้า'

In [ ]:
import mlflow.pytorch
import mlflow
mlflow.set_tracking_uri('http://127.0.0.1:5000')  # set up connection
mlflow.set_experiment('multilabel_model')
# mlflow.transformers.autolog()
with mlflow.start_run() as run:
    # mlflow.transformers.save_model(
    #     transformers_model=pipeline,
    #     path="transformer_pipeline",
    # )
    mlflow.transformers.log_model(
        transformers_model=pipeline,
        artifact_path="multilabel_pipeline",
        
    )
model_uri = "runs:/{}/multilabel_pipeline".format(run.info.run_id)

loaded = mlflow.transformers.load_model(model_uri)
loaded


C:\Users\sern7\AppData\Local\Temp\ipykernel_19388\4265276266.py:11: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.28.1``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  mlflow.transformers.log_model(
c:\Users\sern7\anaconda3\envs\pytorch\Lib\site-packages\mlflow\models\model.py:553: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.28.1``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
2023/05/18 12:41:01 WARNING mlflow.transformers: The model card could not be retrieved from the hub due to [Errno 13] Permission denied: 'multilabel_pretrained'
2023/05/18 12:41:01 WARNING mlflow.tra

In [ ]:
loaded.predict(text)

[{'label': 'LABEL_5', 'score': 0.9974890947341919}]

In [ ]:
from mlflow.models.signature import infer_signature
from mlflow.transformers import generate_signature_output
text = 'ไม่มีที่วางขยะรอจัดเก็บ วางไว้บนทางเท้า'
df = pd.DataFrame([text],columns=['text'])
output =  generate_signature_output(loaded,df)
signature = infer_signature(df, output)



In [ ]:
mlflow.transformers.save_model(
    transformers_model=loaded ,
    path="text-class",
    signature=signature,
    input_example=df,
)

loaded = mlflow.transformers.load_model("text-class")

C:\Users\sern7\AppData\Local\Temp\ipykernel_19388\797863787.py:1: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.28.1``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  mlflow.transformers.save_model(
2023/05/18 12:41:29 WARNING mlflow.transformers: The model card could not be retrieved from the hub due to [Errno 13] Permission denied: 'C:\\Users\\sern7\\AppData\\Local\\Temp\\tmpeytkukox\\multilabel_pipeline\\pipeline'
C:\Users\sern7\AppData\Local\Temp\ipykernel_19388\797863787.py:8: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.28.1``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  loaded = mlflow.transformers.load_model("text-class")


In [ ]:
mlflow.transformers.log_model(
        transformers_model=loaded,
        artifact_path="multilabel_pipeline",
        
    )

C:\Users\sern7\AppData\Local\Temp\ipykernel_19388\3407490784.py:1: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.28.1``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  mlflow.transformers.log_model(
c:\Users\sern7\anaconda3\envs\pytorch\Lib\site-packages\mlflow\models\model.py:553: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.28.1``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
2023/05/18 12:41:36 WARNING mlflow.transformers: The model card could not be retrieved from the hub due to [Errno 13] Permission denied: 'D:\\CU\\Y3T2\\DS\\Data_TraffyFondue\\text-class\\pipeline'


In [ ]:
# import mlflow.pytorch
# import mlflow
# mlflow.set_tracking_uri('http://127.0.0.1:5000')  # set up connection
# mlflow.set_experiment('multilabel-experiment')
# with mlflow.start_run() as run:
#     mlflow.pytorch.log_model(model, "model")
# model_uri = "runs:/{}/model".format(run.info.run_id)
# model = mlflow.pytorch.load_model(model_uri)
# model








In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
   
BERT_MODEL_NAME = "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = CamembertTokenizerFast.from_pretrained(BERT_MODEL_NAME)

In [ ]:
text = 'ไม่มีที่วางขยะรอจัดเก็บ วางไว้บนทางเท้า'

In [ ]:
encoded_input  = tokenizer.encode_plus(text, padding='max_length', max_length=256, truncation=True, return_tensors='pt')

In [ ]:
encoded_input.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:

model(encoded_input['input_ids'])

SequenceClassifierOutput(loss=None, logits=tensor([[-2.8791,  0.3273, -7.1046, -6.1185, -6.3318,  3.3170, -5.5375, -7.7529,
         -8.0882, -7.0107, -8.0564, -8.9456, -7.5059, -8.0857, -6.1525, -7.6930,
         -7.5997, -8.0280, -8.2484, -8.6904, -7.5230, -7.6227, -8.2611, -9.7334]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:




def predict_json(server_url, input_json):
    response = requests.post(server_url, json=input_json)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Request failed with status code: %s, response: %s"
                        % (response.status_code, response.text))
    
def predict(server_url, df):
    data = {"dataframe_split": df.to_dict(orient='split')}
    print(data)
    
    
    
    return predict_json(server_url, data)

text = 'ไม่มีที่วางขยะรอจัดเก็บ วางไว้บนทางเท้า'

# def predict(text):
#     input = tokenizer(text,padding='max_length', max_length = 256, truncation=True,return_tensors="pt")
#     input = input['input_ids'].squeeze(1).to(device)
#     output = model(input).logits
#     output = F.sigmoid(output)
#     res = output.detach().cpu().numpy()
    
#     return output

predict("http://127.0.0.1:1245/invocations", df)

{'dataframe_split': {'index': [0], 'columns': ['text'], 'data': [['ไม่มีที่วางขยะรอจัดเก็บ วางไว้บนทางเท้า']]}}


{'predictions': 'LABEL_5'}

In [ ]:
predict("http://127.0.0.1:1245/invocations", df)

{'dataframe_split': {'index': [0], 'columns': ['text'], 'data': [['ไม่มีที่วางขยะรอจัดเก็บ วางไว้บนทางเท้า']]}}


{'predictions': 'LABEL_5'}